<a href="https://colab.research.google.com/github/Aravinda89/Pytorch-Tutorial/blob/main/Differential_Learning_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning - Differential Learning Rate


## Fine Tuning

In [6]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
testset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


Extracting /root/.pytorch/CIFAR10/cifar-10-python.tar.gz to /root/.pytorch/CIFAR10


In [10]:
for images, labels in trainloader:
  print(images.size(), labels.size())
  break

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [11]:
model = models.vgg16(pretrained=True)
model.classifier

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [12]:
for param in model.parameters():
  param.requires_grad = False

In [13]:
for i in range(0,7):
  model.classifier[i].requires_grad = True

In [14]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096,512),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1)
                      )


In [15]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
criterion = nn.NLLLoss()

In [18]:
from torch.optim import Adam

model = model.to(device)

## Training from the Fully Connected Network onwards

### Re-training the model

In [19]:
from torch.optim import Adam

lr = 3e-4
optimizer = Adam([
    { 'params': model.classifier[0].parameters(), 'lr': lr},
    { 'params': model.classifier[3].parameters(), 'lr': lr},
    { 'params': model.classifier[6].parameters(), 'lr': lr}
    ], lr=lr)

In [20]:
model = model.to(device)
#optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 2.3653225898742676
Epoch(0/1 : Batch number(2/782) : Batch loss : 2.273170232772827
Epoch(0/1 : Batch number(3/782) : Batch loss : 2.2001216411590576
Epoch(0/1 : Batch number(4/782) : Batch loss : 2.0917232036590576
Epoch(0/1 : Batch number(5/782) : Batch loss : 1.9808127880096436
Epoch(0/1 : Batch number(6/782) : Batch loss : 1.984837532043457
Epoch(0/1 : Batch number(7/782) : Batch loss : 1.9999157190322876
Epoch(0/1 : Batch number(8/782) : Batch loss : 1.884743094444275
Epoch(0/1 : Batch number(9/782) : Batch loss : 1.7808301448822021
Epoch(0/1 : Batch number(10/782) : Batch loss : 1.7544889450073242
Epoch(0/1 : Batch number(11/782) : Batch loss : 1.6620538234710693
Epoch(0/1 : Batch number(12/782) : Batch loss : 1.6983424425125122
Epoch(0/1 : Batch number(13/782) : Batch loss : 1.373589038848877
Epoch(0/1 : Batch number(14/782) : Batch loss : 1.4959750175476074
Epoch(0/1 : Batch number(15/782) : Batch loss : 1.3911482095718384
Epoch(0/

KeyboardInterrupt: ignored

### The accuracy of the model

In [21]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 71.875% 


## Un-freezing & training on the LAST CNN block onwards

### Re-training the model

In [ ]:
for i in range(24,31):
  model.features[i].requires_grad = True


In [ ]:
from torch.optim import Adam

lr = 3e-4
optimizer = Adam([
    { 'params': model.features[24].parameters(), 'lr': lr/3},
    { 'params': model.features[26].parameters(), 'lr': lr/3},
    { 'params': model.features[28].parameters(), 'lr': lr/3},
    { 'params': model.classifier[0].parameters(), 'lr': lr},
    { 'params': model.classifier[3].parameters(), 'lr': lr},
    { 'params': model.classifier[6].parameters(), 'lr': lr}
    ], lr=lr)

In [ ]:
model = model.to(device)
#optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

### The accuracy of the model

In [ ]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

## Un-freezing & training on the LAST TWO CNN block onwards

### Re-training the model

In [22]:
for i in range(17,24):
  model.features[i].requires_grad = True

In [23]:
from torch.optim import Adam

lr = 3e-4
optimizer = Adam([
    { 'params': model.features[17].parameters(), 'lr': lr/9},
    { 'params': model.features[19].parameters(), 'lr': lr/9},
    { 'params': model.features[21].parameters(), 'lr': lr/9},
    { 'params': model.features[24].parameters(), 'lr': lr/3},
    { 'params': model.features[26].parameters(), 'lr': lr/3},
    { 'params': model.features[28].parameters(), 'lr': lr/3},
    { 'params': model.classifier[0].parameters(), 'lr': lr},
    { 'params': model.classifier[3].parameters(), 'lr': lr},
    { 'params': model.classifier[6].parameters(), 'lr': lr}
    ], lr=lr)

In [24]:
model = model.to(device)
#optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0
  
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()
    
    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')
    
  print(f'Training loss : {batch_loss/len(trainloader)}')  
    

Epoch(0/1 : Batch number(1/782) : Batch loss : 0.6579272150993347
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.7019481658935547
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.9410093426704407
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.7407578825950623
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.770391583442688
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.909604549407959
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.5332302451133728
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.6948360800743103
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.5841999053955078
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.7174786329269409
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.8692085146903992
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.8495988249778748
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.7417474985122681
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.745541512966156
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.6796875596046448
Epoch(0

### The accuracy of the model

In [25]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):
        
        logps = model(images)
        output = torch.exp(logps)
        
        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')
        
        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 83.125% 
